# Juan Manuel Deutsch, Cesar Felipe Giraldo, Julian Felipe Pulido

# Exercise 1

## 1.1)


Para este ejercicio tenemos $X$ siendo la matriz base y $Σ$ como la matriz de covarianza de $X$

Tambien tenemos los componentes principales $u_1, u_2,...,u_n$, que son los autovectores o vectores propios de $Σ$ que tienen varianzas asociadas $u_i^TΣu_i$, donde $u_i^T$ es la transpuesta del vector propio $u_i$

Ahora tendremos una tranformación lineal de los datos $X$, siendo $AX$ donde $A$ es una transformación $m x m$

Ahora con la matriz transformada tenemos la matriz de covarianza transformada $AΣA^T$ cuyos componentes principales estan dados por $v_1, v_2,...,v_n$ estos siendo los autovectores cuyas varianza asociadas estan dadas por $v_i^TAΣA^Tv_i$

Teniendo en cuenta que necesitamos demostrar los componentes principales y como cambian tenemos con relacion a $AX$ y $X$

$u_1$ siendo componente principal cuya varianza asociada es $u_i^TΣu_i$ y siendo $v_1$ el componente principal cuya varianza asociada es $v_i^TAΣA^Tv_i$

Teniendo en cuenta que $u_1$ y $v_1$ estan relacionados por una transformación lineal tenemos que $v_1 = Tu_1$ para una matriz T

Ahora, se puede escribir la varianza asociada de $v_1$ como:

$$v_1^TAΣA^Tv_1 = (Tu_1)^TAΣA^T(Tu_1) = u_1^TT^TAΣA^TTu_1$$

Dado que $T$ es una matriz de transformacion lineal, $T^TAΣA^TT$ es otra matriz de covarianza $Σ'$

Por lo tanto ahora tenemos:

$$v_1^TAΣA^Tv_1 = u_1^TΣ'u_1$$

Concluyendo con esta demostración, tenemos que la varianza asociada $v_1$ depende de la matriz de covarianza $Σ'$ que es una version transformada de la matriz de covarianza original $Σ$, por lo cual, $Σ'$ es diferente a $Σ$, entonces la varianza asociada a estos van a ser diferentes, por lo que un valor propio esta ligado conjunto de datos original y el otro valor propio esta ligado al conjunto transformado

## 1.2

Para esta demostracion tenemos $D$, como una matriz $f x c$ donde $f$ son filas y $c$ son columnas, ademas de $D'$ que es la matriz transformada con una transformación afin a una o mas columnas

Iniciando calculamos la matriz de correlación $corr(D)$
$$corr(D) = 1/n (D - D_u)^T(D - D_u)$$
Donde $D_u$ es la media de cada columna de $D$
De acuerdo a eso se los componentes principales por medio de $corr(D)$

Ahora realizamos el mismo proceso con la tranformación afin, que seria del siguiente modo:

$$D' = A*D+b$$

Donde $A$ es la transformación lineal y $b$ es el desplazamiento

Ahora calculamos la correlacion 

$$corr(D') = 1/n (D' - D'_u)^T(D' - D'_u)$$

Donde ahora $D'_u$ es la media de la matriz transformada, donde las columnas de $D'$ se ven afectadas por la transformación afin

Pero debemos tener en cuenta que para la covarianza los efectos de una transformación afin se dan de la siguiente forma:

$$cov(D') = A^Tcov(D)A$$

No obstante, estamos tratando es con matrices de correlación, las cuales son versiones escaladas de las matrices de covariaza, por lo cual la matriz de covarianza tambien afecta a la matriz de correlación con la misma transformación afin

Por lo cual los autovectores o vectores propios de $corr(D')$ son los mismos que $corr(D)$

# Exercise 2

### 2.1)

In [3]:

import numpy as np

def PCA(D, red):
    mu = np.mean(D, axis=0)
    Z = D - mu
    cov_matrix = np.cov(Z, rowvar=False, bias=False)
    eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)
    
    sorted_indices = np.argsort(eigenvalues)[::-1]

    topEigenvalues = eigenvalues[sorted_indices]
    topEigenVector = eigenvectors[:,sorted_indices]
    cumvariance = np.cumsum(topEigenvalues)

    if red < 1:
        total_variance = np.sum(topEigenvalues)
        cumvariance = np.cumsum(topEigenvalues)
        variance_top = red * total_variance
        numdims = np.argmax(cumvariance >= variance_top) + 1
        
    else:
        numdims = red
    top_eigenvectors = topEigenVector[:, :numdims] 
    total_variance = np.sum(topEigenvalues[:numdims]) 
    variance = total_variance / np.sum(topEigenvalues)
    A = np.dot(Z, top_eigenvectors)


    v = np.sum(topEigenvalues[:numdims])

    
    return A, v

D = np.array([[4, 1, 0, 9, 8],
 [9, 9, 1, 6, 1],
 [4, 5, 3, 3, 5],
 [6, 2, 7, 3, 8],
 [5, 1, 0, 8, 1],
 [0, 4, 4, 6, 3],
 [3, 3, 7, 6, 3],
 [0, 6, 9, 3, 4],
 [4, 5, 0, 1, 6],
 [6, 5, 1, 3, 1]])

A, variance = PCA(D, 4)
print(f'Datos reducidos A:\n {A}\n')
print(f'La varianza de la matriz A: {variance}')

Datos reducidos A:
 [[-2.22383293  6.62077922 -0.9864382   0.03164106]
 [-5.55127862 -4.7594466   0.5427392   2.28080035]
 [ 0.55058985 -0.98729531 -1.77065664 -0.80993837]
 [ 3.68276669  1.36321497 -3.73620713  3.34999953]
 [-4.37500465  2.74874794  3.09709231  0.07380571]
 [ 2.10915442  0.84383508  2.99845749 -2.2964254 ]
 [ 3.03064114  0.41677691  2.57421906  1.72366106]
 [ 6.92241058 -2.45359706  1.51927006 -0.63565965]
 [-1.04675051 -0.9551932  -4.22672217 -3.08842427]
 [-3.09869596 -2.83782195 -0.01175398 -0.62946002]]

La varianza de la matriz A: 37.02020528711315


### 2.2)

##### Preparación de datos

In [1]:
import pandas as pd
import numpy as np
import time

iris_data = pd.read_csv('iris.csv')
baseball_data = pd.read_csv('baseball_numeric.csv')
fabert_data = pd.read_csv('fabert.csv')
amazon_data = pd.read_csv('amazon.csv')


adf2 = amazon_data.drop('Class', axis=1).values
fdf2 = fabert_data.values
idf2 = iris_data.drop('species', axis=1).values
bndf = baseball_data.values

datasets = [bndf, fdf2, idf2]
data_dict = ['Baseball Dataset', 'Fabert Dataset', 'Iris Dataset']

datasetsDictionary = {'iris': iris_data, 'baseball_numeric': baseball_data, 'fabert': fabert_data}


/var/folders/4k/4b01rf9j7cl6mcd3652m260w0000gn/T/ipykernel_87687/3708956956.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


##### a)

In [3]:


def getStats(A):

        mu = list(sum(row)/len(row) for row in zip(*A))

        n = A.shape[0]
        cov_mu = A - mu
        cov = np.dot(cov_mu.T, cov_mu) / (n)
 
        desv = np.sqrt(np.diag(cov))
        cor = cov / np.outer(desv, desv)

        return mu, cov, cor
results = []

for dataset_name, dataset in datasets.items():

    numeric_cols = dataset.select_dtypes(include=[np.number]).columns
    dataset_numeric = dataset[numeric_cols]
    
    A = dataset_numeric.values.astype(float)  
    
    start_time_getStats = time.time()
    mu, cov, cor = getStats(A)
    getStats_time = time.time() - start_time_getStats
    
    start_time_np = time.time()
    cov_np = np.cov(A, rowvar=False, bias=False)
    desv = np.sqrt(np.diag(cov_np))
    cor_np = cov_np / np.outer(desv, desv)
    np_time = time.time() - start_time_np
    
    results.append({'Dataset': dataset_name, 'Tiempo getStats': getStats_time, 'Tiempo Numpy': np_time})


results_df = pd.DataFrame(results)
print(results_df)

AttributeError: 'list' object has no attribute 'items'

In [27]:
import timeit
import numpy as np
import pandas as pd



def getStats(A):
    start = timeit.default_timer()
    #Media
    mu = list(sum(row)/len(row) for row in zip(*A))
    #Covarianza
    n = A.shape[0]
    cov_mu = A - mu
    cov = np.dot(cov_mu.T, cov_mu) / (n)
    return mu, cov, timeit.default_timer() - start

def getStatsNumpy(A):
    start = timeit.default_timer()
    mu = np.mean(A, axis=0)
    cov = np.cov(A, rowvar=False, bias=False)
    return mu, cov, timeit.default_timer() - start


all_results = []

for dataset in datasets:
    combined_results = np.array([getStats(dataset)[2], getStatsNumpy(dataset)[2]]) 
    all_results.append(combined_results)

df = pd.DataFrame(all_results, index = data_dict, columns=['Nuestra función', 'Función de numpy'])
display(df) 

,Nuestra función,Función de numpy
Amazon Dataset,5.521663,4.341354
Baseball Dataset,0.006432,0.000730
Fabert Dataset,0.805461,0.299242
Iris Dataset,0.000289,0.000200


##### b)

In [6]:
import numpy as np
import pandas as pd
results = []

for dataset_name, dataset in datasetsDictionary.items():
    
    numeric_dataset = dataset.select_dtypes(include=[np.number])
    A = numeric_dataset.values.astype(float)

    reduced_data, variance = PCA(A, 0.9)
    
    total_variance_original = np.sum(np.linalg.eigvals(np.cov(A, rowvar=False)))
    relative_total_variance = variance / total_variance_original
    
    results.append({'Dataset': dataset_name, 'Dimensiones': reduced_data.shape[1], 'Varianza total absoluta': variance, 'Varianza Total Relativa': relative_total_variance})


results_df = pd.DataFrame(results)
print(results_df)

            Dataset  Dimensiones  Varianza total absoluta  \
0              iris            1                 4.224841   
1  baseball_numeric            2             17228.155686   
2            fabert          391                 5.141834   

   Varianza Total Relativa  
0                 0.924616  
1                 0.989389  
2                 0.900184  


##### c)

In [79]:

results = []
for dataset_name, dataset in datasetsDictionary.items():

    numeric_dataset = dataset.select_dtypes(include=[np.number])
    
    X = numeric_dataset.values.astype(float)
    
    variances = []
    relative_variances = []
    
    for n_components in [1, 2, 3]:

        reduced_data, total_variance = PCA(X, n_components)
        relative_total_variance = total_variance / np.sum(np.linalg.eig(np.cov(X, rowvar=False, bias=False))[0])
        
        variances.append(total_variance)
        relative_variances.append(relative_total_variance)
    
    results.append({'Dataset': dataset_name, 'Varianza (1D)': variances[0], 'Varianza Relativa (1D)': relative_variances[0],
                    'Varianza (2D)': variances[1], 'Varianza Relativa(2D)': relative_variances[1],
                    'Varianza (3D)': variances[2], 'Varianza Relativa (3D)': relative_variances[2]})

results_df = pd.DataFrame(results)
results_df = results_df.round(3)  
print(results_df)

            Dataset  Varianza (1D)  Varianza Relativa (1D)  Varianza (2D)  \
0              iris          4.225                   0.925          4.467   
1  baseball_numeric      11828.850                   0.679      17228.156   
2            fabert          2.928                   0.513          2.953   

   Varianza Relativa(2D)  Varianza (3D)  Varianza Relativa (3D)  
0                  0.978          4.546                   0.995  
1                  0.989      17391.761                   0.999  
2                  0.517          2.975                   0.521  
